# Prompt Templates

Learn how to create, save, and reuse prompt templates for consistent and efficient prompt engineering.

## What You'll Learn
- Creating templates with variable substitution
- Saving and loading templates
- Using templates for consistent prompts
- Template validation and best practices

In [ ]:
from prompt_playground.client import create_client, send_prompt
from prompt_playground.prompts import PromptTemplate, PromptLibrary, validate_prompt
from rich import print as rprint
from rich.panel import Panel
from rich.table import Table

client = create_client()
rprint("[green]✓[/green] Setup complete")

## Why Use Templates?

Templates help you:
- **Maintain consistency** across similar prompts
- **Save time** by reusing proven patterns
- **Version control** your prompts
- **Scale** prompt engineering across teams

## Creating Your First Template

Templates use `{variable}` syntax for placeholders.

In [ ]:
template = PromptTemplate(
    template="Explain {topic} to a {audience} in {style} language.",
    variables=["topic", "audience", "style"]
)

rprint("[green]✓[/green] Template created")
rprint(f"Variables: {template.variables}")

## Using Templates

Fill in variables using the `fill()` method:

In [ ]:
filled_prompt = template.fill(
    topic="machine learning",
    audience="10-year-old",
    style="simple"
)

rprint(Panel(filled_prompt, title="[bold cyan]Filled Prompt[/bold cyan]", expand=False))

response = send_prompt(prompt=filled_prompt, client=client)
rprint(Panel(response['text'], title="[bold green]Response[/bold green]", expand=False))

## Template Validation

Ensure your template variables match the placeholders:

In [ ]:
valid_template = PromptTemplate(
    template="Write a {length} {type} about {subject}.",
    variables=["length", "type", "subject"]
)

invalid_template = PromptTemplate(
    template="Write a {length} {type} about {subject}.",
    variables=["length", "type"]
)

rprint(f"Valid template: {valid_template.validate()}")
rprint(f"Invalid template: {invalid_template.validate()}")

## Saving Templates to Library

Use `PromptLibrary` to persist templates:

In [ ]:
library = PromptLibrary('my_templates.json')

explanation_template = PromptTemplate(
    template="Explain {concept} using a {analogy_type} analogy.",
    variables=["concept", "analogy_type"]
)

library.save('explanation', explanation_template)
rprint("[green]✓[/green] Template saved to library")

## Loading and Using Saved Templates

In [ ]:
loaded_template = library.load('explanation')

prompt = loaded_template.fill(
    concept="blockchain",
    analogy_type="everyday object"
)

response = send_prompt(prompt=prompt, client=client)
rprint(Panel(response['text'], title="[bold]Response from Loaded Template[/bold]", expand=False))

## Building a Template Collection

Create templates for common use cases:

In [ ]:
templates_to_create = {
    'summarize': PromptTemplate(
        template="Summarize the following text in {num_sentences} sentences:\n\n{text}",
        variables=["num_sentences", "text"]
    ),
    'translate': PromptTemplate(
        template="Translate this {source_lang} text to {target_lang}:\n\n{text}",
        variables=["source_lang", "target_lang", "text"]
    ),
    'code_review': PromptTemplate(
        template="Review this {language} code for {focus_area}:\n\n{code}",
        variables=["language", "focus_area", "code"]
    ),
    'email_draft': PromptTemplate(
        template="Write a {tone} email to {recipient} about {subject}.",
        variables=["tone", "recipient", "subject"]
    )
}

for name, template in templates_to_create.items():
    library.save(name, template)

rprint(f"[green]✓[/green] Saved {len(templates_to_create)} templates")

## Listing All Templates

In [ ]:
all_templates = library.list_all()

table = Table(title="Template Library")
table.add_column("Name", style="cyan")
table.add_column("Variables", style="green")
table.add_column("Variable Count", style="yellow")

for name in all_templates:
    template = library.load(name)
    table.add_row(name, ", ".join(template.variables), str(len(template.variables)))

from rich.console import Console
console = Console()
console.print(table)

## Using Templates in Practice

### Example 1: Email Generation

In [ ]:
email_template = library.load('email_draft')

scenarios = [
    {"tone": "professional", "recipient": "manager", "subject": "project deadline extension"},
    {"tone": "friendly", "recipient": "colleague", "subject": "lunch meeting"},
    {"tone": "formal", "recipient": "client", "subject": "proposal submission"}
]

for scenario in scenarios:
    prompt = email_template.fill(**scenario)
    response = send_prompt(prompt=prompt, max_tokens=200, client=client)
    
    rprint(f"\n[bold cyan]{scenario['tone'].title()} to {scenario['recipient']}:[/bold cyan]")
    rprint(Panel(response['text'], expand=False))

### Example 2: Code Review with Templates

In [ ]:
code_template = library.load('code_review')

code_sample = '''def calculate_total(items):
    total = 0
    for item in items:
        total = total + item['price']
    return total'''

prompt = code_template.fill(
    language="Python",
    focus_area="performance and best practices",
    code=code_sample
)

response = send_prompt(prompt=prompt, client=client)
rprint(Panel(response['text'], title="[bold]Code Review[/bold]", expand=False))

## Advanced: Nested Templates

Combine templates for complex prompts:

In [ ]:
base_template = PromptTemplate(
    template="{instruction}\n\nContext: {context}\n\nRequirements: {requirements}",
    variables=["instruction", "context", "requirements"]
)

instruction_template = PromptTemplate(
    template="Create a {output_type} for {target_audience}",
    variables=["output_type", "target_audience"]
)

instruction = instruction_template.fill(
    output_type="technical tutorial",
    target_audience="beginner developers"
)

final_prompt = base_template.fill(
    instruction=instruction,
    context="Teaching Python basics",
    requirements="Include code examples, be concise, use simple language"
)

rprint(Panel(final_prompt, title="[bold]Nested Template Result[/bold]", expand=False))

## Prompt Validation

Check prompt quality before sending:

In [ ]:
test_prompts = [
    "hi",
    "Explain quantum computing in detail with examples and use cases",
    "WRITE ME A STORY ABOUT ROBOTS"
]

for prompt in test_prompts:
    validation = validate_prompt(prompt)
    
    rprint(f"\n[cyan]Prompt:[/cyan] {prompt[:50]}...")
    rprint(f"Valid: {validation['valid']}")
    if validation['issues']:
        rprint(f"[yellow]Issues:[/yellow] {', '.join(validation['issues'])}")
    if validation['suggestions']:
        rprint(f"[blue]Suggestions:[/blue] {', '.join(validation['suggestions'])}")

## Template Management

### Deleting Templates

In [ ]:
test_template = PromptTemplate(
    template="Test template: {var}",
    variables=["var"]
)

library.save('test_temp', test_template)
rprint(f"Templates before delete: {len(library.list_all())}")

library.delete('test_temp')
rprint(f"Templates after delete: {len(library.list_all())}")

## Best Practices

### 1. Use Descriptive Variable Names

In [ ]:
good_template = PromptTemplate(
    template="Analyze {customer_feedback} and identify {num_top_issues} top issues.",
    variables=["customer_feedback", "num_top_issues"]
)

rprint("[green]Good:[/green] Descriptive variable names make templates self-documenting")

### 2. Include Examples in Templates

In [ ]:
format_template = PromptTemplate(
    template="""Convert the input to {output_format} format.

Example:
Input: {example_input}
Output: {example_output}

Now convert:
Input: {actual_input}
Output:""",
    variables=["output_format", "example_input", "example_output", "actual_input"]
)

library.save('format_converter', format_template)

### 3. Version Your Templates

In [ ]:
v1_template = PromptTemplate(
    template="Summarize {text}",
    variables=["text"]
)

v2_template = PromptTemplate(
    template="Summarize {text} in {style} style, focusing on {key_points}",
    variables=["text", "style", "key_points"]
)

library.save('summarize_v1', v1_template)
library.save('summarize_v2', v2_template)

rprint("[green]✓[/green] Templates versioned for iteration")

## Summary

You've learned:
- ✓ Creating templates with variables
- ✓ Saving and loading templates from library
- ✓ Validating templates and prompts
- ✓ Building reusable template collections
- ✓ Advanced techniques (nesting, versioning)
- ✓ Best practices for template management

## Next Steps

- **03_ab_testing.ipynb**: Compare template variations
- **04_batch_processing.ipynb**: Use templates for batch operations
- **05_evaluation_metrics.ipynb**: Evaluate template performance